# Measuring individual amplitudes

Individual amplitudes can be measured up to a global phase.  This tutorial follows Nature Communications 7, Article number: 10439 (2016) doi:10.1038/ncomms10439.

Given a pure state wavefunction
\begin{align}
\vert \psi \rangle = \sum_{j=0}^{d-1}c_{j} \vert j \rangle
\end{align}
where $d$ is the dimension of Hilbert space and $\vert j \rangle$ represents a basis state of the Hilbert space each $c_{j}$ can be measured by defining an operator $C_{j}$
\begin{align}
C_{j} = \vert a \rangle \langle j \vert
\end{align}
where $\vert a \rangle$ is a target computational basis state.  For example, if $\vert a \rangle$ is the all-zero state.  Then the operator $C_{j}$ can be decomposed into tensor products of operators $\{\vert 0 \rangle \langle 1 \vert, \vert 1 \rangle \langle 0 \vert, \vert 0 \rangle \langle 0 \vert, \vert 1 \rangle \langle 1 \vert \}$ each of which can be decomposed into the following:
\begin{align}
\vert 0 \rangle \langle 0 \vert =  \frac{\sigma^{z} + I}{2} \\
\vert 1 \rangle \langle 1 \vert =  \frac{I - \sigma^{z}}{2} \\
\vert 0 \rangle \langle 1 \vert =  \frac{1}{2}\left( \sigma^{x} + i \sigma^{y}\right)\\
\vert 1 \rangle \langle 0 \vert =  \frac{1}{2}\left( \sigma^{x} - i \sigma^{y}\right).
\end{align}
The expected value for each $C_{j}$ corresponds to the amplitude multiplied by an amplitude independent measurement
\begin{align}
\langle \psi \vert C_{j} \vert \psi \rangle = \langle \psi \vert a \rangle c_{j}
\end{align}
which corresponds to a complex weighted sum of Pauli operators.  Each term in the non-Hermitian operator can be evaluated by operator averaging to fixed precision.  Therefore, each amplitude corresponds to 
\begin{align}
c_{j} = \frac{1}{v}\sum_{q}\langle O_{j, q} \rangle 
\end{align}
where $O_{j, q}$ is the $q$-th term in the operator expansion for $C_{j}$ and $v$ is $\sqrt{|\langle \psi \vert a \rangle|^{2}}$.   Therefore, we can measure an amplitude with respect to the phase of a reference state. 

Below is an example measuring the state generated by a random 2-qubit unitary.  First we have some functions for generating random 1- and 2-qubit unitaries.

In [1]:
import numpy as np
from scipy.linalg import expm

Xmat = np.array([[0, 1], [1, 0]])
Ymat = np.array([[0, -1j], [1j, 0]])
Zmat = np.array([[1, 0], [0, -1]])
Imat = np.array([[1, 0], [0, 1]])


def generate_random_one_qubit_unitary():
    """
    Generate anything from U(2)
    """
    a, b, c, d = np.random.random(4) * 2 * np.pi  # random samples [0, 2 pi )
    unitary = expm(-1j*Imat*a)
    unitary = expm(-1j*Zmat*b).dot(unitary)
    unitary = expm(-1j*Ymat*c).dot(unitary)
    unitary = expm(-1j*Zmat*d).dot(unitary)
    return unitary

def generate_random_two_qubit_unitary():
    """
    Generate something from U(4)
    """
    group_params = np.random.random(3) * 2 * np.pi  # random samples from [0, 2pi)
    su2_group_1_1 = generate_random_one_qubit_unitary()
    su2_group_1_2 = generate_random_one_qubit_unitary()
    su2_group_2_1 = generate_random_one_qubit_unitary()
    su2_group_2_2 = generate_random_one_qubit_unitary()

    xx_mat = np.kron(Xmat, Xmat)
    yy_mat = np.kron(Ymat, Ymat)
    zz_mat = np.kron(Zmat, Zmat)
    entanglers = expm(-1j * (group_params[0] * xx_mat +
                             group_params[1] * yy_mat +
                             group_params[2] * zz_mat))
    su2_group_1 = np.kron(su2_group_1_1, su2_group_1_2)
    su2_group_2 = np.kron(su2_group_2_1, su2_group_2_2)

    unitary = su2_group_2.dot(entanglers).dot(su2_group_1)
    return unitary


Now use grove to estimate a state generated by a 2-qubit random unitary.  We compare the density matrix elementwise because we can only measure the state up to a global phase which is set by the reference amplitude being real valued.

In [3]:
from itertools import product
from pyquil.api import QVMConnection
from grove.measurements.amplitude_measurement import  measure_pure_state
from grove.alpha.arbitrary_state.arbitrary_state import create_arbitrary_state
quantum_resource = QVMConnection()

initial_state = np.array([[1], [0], [0], [0]])
num_qubits = int(np.log2(initial_state.shape[0]))
np.random.seed(42)

random_unitary = generate_random_two_qubit_unitary()
state = random_unitary.dot(initial_state)
state_prep_prog = create_arbitrary_state(state.flatten())
rho_true = state.dot(np.conj(state).T)
wavefunction = measure_pure_state(state_prep_prog, [0] * num_qubits,
                                  quantum_resource=quantum_resource,
                                  variance_bound=1.0E-4)
rho_test = wavefunction.dot(np.conj(wavefunction).T)

for ii, jj in product(range(rho_test.shape[0]), repeat=2):
    print(rho_test[ii, jj], rho_true[ii, jj], rho_test[ii, jj] - rho_true[ii, jj])

norm = np.linalg.norm(rho_test - rho_true)
print(norm)

(0.105505674482+0j) (0.117550279387-2.53206738822e-18j) (-0.0120446049051+2.53206738822e-18j)
(0.236244648318+0.106786391437j) (0.242994581764+0.117881653987j) (-0.00674993344641-0.0110952625494j)
(0.072873853211+0.0754458715596j) (0.0749923322931+0.0858658856063j) (-0.00211847908212-0.0104200140467j)
(-0.0573464831804+0.116217125382j) (-0.0544658028107+0.121763854242j) (-0.00288068036969-0.00554672885988j)
(0.236244648318-0.106786391437j) (0.242994581764-0.117881653987j) (-0.00674993344641+0.0110952625494j)
(0.637073480505-4.20826128067e-18j) (0.620521290923-1.36830864372e-17j) (0.0165521895816+9.47482515655e-18j)
(0.239538302752+0.0951773222477j) (0.241128674345+0.102294310625j) (-0.00159037159242-0.00711698837773j)
(-0.0107803899083+0.318271961009j) (0.00951788090363+0.306324033805j) (-0.0202982708119+0.0119479272047j)
(0.072873853211-0.0754458715596j) (0.0749923322931-0.0858658856063j) (-0.00211847908212+0.0104200140467j)
(0.239538302752-0.0951773222477j) (0.241128674345-0.10229431